# Vodaphone: Close ports after sent!

Load Libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import os

Get all files in KPN directory

In [2]:
def get_files_in_directory(directory):
    # Initialize an empty list to store file names
    files = []

    # Loop through all files and directories in the specified directory
    for file_name in os.listdir(directory):
        # Check if the current entry is a file
        if os.path.isfile(os.path.join(directory, file_name)):
            # If it's a file, add its name to the list
            files.append(file_name)

    return files

directory = "data/vodaphone_nl/"
files_in_directory = get_files_in_directory(directory)
print("Files in directory:", files_in_directory)

Files in directory: ['Vodafone_LONG_PHONE_6.csv', 'Vodafone_LONG_PHONE_multiple_10.csv', 'data_Vodaphone_server_5.csv', 'data_Vodaphone_server_1.csv', 'data_Vodafone_server_10.csv', 'data_Vodaphone_server_7.csv', 'Vodafone_LONG_PHONE_multiple_9.csv', 'data_Vodaphone_server_4.csv', 'Vodafone_LONG_PHONE_5.csv', 'Vodafone_LONG_PHONE_multiple_8.csv', 'Vodafone_LONG_PHONE_7.csv', 'Vodafone_LONG_PHONE_3.csv', 'Vodafone_LONG_PHONE_4.csv', 'data_Vodaphone_server_8.csv', 'data_Vodafone_server_3.csv', 'data_Vodafone_server_9.csv', 'Vodafone_LONG_PHONE_2.csv', 'data_Vodaphone_server_6.csv', 'Vodafone_LONG_PHONE_1.csv', 'data_Vodaphone_server_2.csv']


Load file 1 and visualize 

In [ ]:
# Load data and clean up

df_phone = pd.read_csv("data/vodaphone_nl/Vodafone_LONG_PHONE_1.csv")
df_server = pd.read_csv("data/vodaphone_nl/data_Vodaphone_server_1.csv")
trimmed_columns = {col: col.strip() for col in df_server.columns}
df_server = df_server.rename(columns=trimmed_columns)
trimmed_columns = {col: col.strip() for col in df_phone.columns}
df_phone = df_phone.rename(columns=trimmed_columns)
df_merged = df_phone.merge(df_server, how="inner", on="Message id")
df_merged.sort_values(by="Time_y")
df_merged["Time_y"] = df_merged["Time_y"].astype(int)
df_merged["time_difference_y"] = df_merged['Time_y'].diff()
df_merged.sort_values(by="Time_x")
df_merged["Time_x"] = df_merged["Time_x"].astype(int)
df_merged["time_difference_x"] = df_merged['Time_x'].diff()
df_merged["Sender Port_x"] = df_merged["Sender Port_x"].astype(str)
df_merged["Sender Port_y"] = df_merged["Sender Port_y"].astype(str)


print(f"Number of rows {len(df_merged.index)}")
# Visualize 

# Create a line chart using Plotly
fig = go.Figure()

# Add a line trace
fig.add_trace(go.Scatter(x=df_merged['Time_x'], y=df_merged['Sender Port_y'], mode='lines', name='Port_y'))

# Update layout
fig.update_layout(title='Line Chart of Time and Value for set 1',
                  xaxis_title='Time',
                  yaxis_title='Value')

# Show the plot
fig.show()

In [3]:
df = pd.DataFrame()
for i in range(1,11):
    names = list(filter(lambda x: f"_{i}.csv" in x, files_in_directory))
    phone = list(filter(lambda x: "PHONE" in x, names))[0]
    server = list(filter(lambda x: "server" in x, names))[0]
    df_phone = pd.read_csv(f"data/vodaphone_nl/{phone}")
    df_server = pd.read_csv(f"data/vodaphone_nl/{server}")
    trimmed_columns = {col: col.strip() for col in df_server.columns}
    df_server = df_server.rename(columns=trimmed_columns)
    trimmed_columns = {col: col.strip() for col in df_phone.columns}
    df_phone = df_phone.rename(columns=trimmed_columns)
    df_merged = df_phone.merge(df_server, how="inner", on="Message id")
    df_merged.sort_values(by="Time_y")
    df_merged["Time_y"] = df_merged["Time_y"].astype(int)
    df_merged["time_difference_y"] = df_merged['Time_y'].diff()
    df_merged.sort_values(by="Time_x")
    df_merged["Time_x"] = df_merged["Time_x"].astype(int)
    df_merged["time_difference_x"] = df_merged['Time_x'].diff()
    df_merged["Sender Port_x"] = df_merged["Sender Port_x"].astype(int)
    df_merged["Sender Port_y"] = df_merged["Sender Port_y"].astype(int)
    df_merged["label"] = i
    df_merged = df_merged[["Time_x", 'Sender Port_x', 'Sender Port_y', "label"]]
    print(len(df_merged.index))
    df = pd.concat([df, df_merged], ignore_index=True)

df

39571
0
46255
12068
35433
33448
10664
257946
386895
92855


,Time_x,Sender Port_x,Sender Port_y,label
0,1708342929936,21996,15084,1
1,1708342930363,23667,15091,1
2,1708342930786,48265,15081,1
3,1708342931209,26006,15094,1
4,1708342931631,37716,15092,1
...,...,...,...,...
915130,1709544860375,37034,11690,10
915131,1709544860818,54713,11705,10
915132,1709544861261,47957,11701,10
915133,1709544861705,57050,11706,10


In [ ]:
len(df['Sender Port_y'].unique())

### Vodaphone' ~915k lines of data mmaps only to 10.8k ports??????

Find for how long it goes linear? 

In [ ]:
l = [[0]]
THRESHOLD = 1
prev = df.iloc[0]
for index, row in df.iterrows():
    

    if abs(row["Sender Port_y"] - prev["Sender Port_y"]) > THRESHOLD or row["Time_x"] - prev["Time_x"] > 10000:
        l[-1].append(index -1)
        l.append([index])


    prev = row
l.pop()
print(l)
print(len(l))

In [ ]:
differences = [ e-b  for b,e in l ]
print(differences)
print(len(differences))

In [ ]:
print(len(differences))

## WORKS DIFFERENT THAN THE REST, SOME PORTS ARE RE USED ALSO NOT SURPER LINEAR!

In [4]:
df[["Sender Port_x","Sender Port_y"]].value_counts()

Sender Port_x  Sender Port_y
18647          5047             4
20930          7298             4
44557          9005             4
48650          6634             4
14599          8583             4
                               ..
22823          12487            1
               15143            1
22824          5032             1
               6152             1
65535          11999            1
Name: count, Length: 888969, dtype: int64

In [ ]:
df["Sender Port_y"].value_counts()

In [11]:
data = df['Sender Port_y']
density, bins = np.histogram(data, bins=10000, density=True)
midpoints = (bins[1:] + bins[:-1]) / 2

# Create line plot of the PDF using Plotly
fig = go.Figure(data=[go.Scatter(x=midpoints, y=density, mode='lines')])
fig.update_layout(title='Probability Density Function of mapped ports (10000 bins)', xaxis_title='Value', yaxis_title='Density')
fig.show()

In [9]:
data = df['Sender Port_y']
density, bins = np.histogram(data, bins=10, density=True)
midpoints = (bins[1:] + bins[:-1]) / 2

# Create line plot of the PDF using Plotly
fig = go.Figure(data=[go.Scatter(x=midpoints, y=density, mode='lines')])
fig.update_layout(title='Probability Density Function of mapped ports (10 bins)', xaxis_title='Value', yaxis_title='Density')
fig.show()